# Fine-tune NER Model for Amharic Telegram Messages

- The notebook demonstrates the fine-tuning of a BERT multilingual model for Named Entity Recognition (NER) on Ethiopian market data. 

## 1. Environment Setup

In [1]:
# Install required packages
import os
import random
import numpy as np
import torch

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

def set_seed(seed: int = 42):
    """Set random seed for reproducibility"""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)

from transformers import (
    AutoTokenizer, 
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForTokenClassification
)
from datasets import Dataset, DatasetDict
import evaluate
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score

## 2. Load & Rebalance CoNLL Data

In [2]:
from datasets import Dataset
from collections import Counter


def read_conll(filepath):
    data = []
    tokens, ner_tags = [], []
    with open(filepath, encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    data.append({"tokens": tokens, "ner_tags": ner_tags})
                    tokens, ner_tags = [], []
            else:
                token, tag = line.split()[0], line.split()[-1]
                tokens.append(token)
                ner_tags.append(tag)
        if tokens:
            data.append({"tokens": tokens, "ner_tags": ner_tags})
    return data

conll_path = "../data/labeled/auto_ner_dataset.conll"
examples = read_conll(conll_path)

# Optional Rebalancing Step
entity_counts = Counter(tag for ex in examples for tag in ex["ner_tags"] if tag != 'O')
print("Original entity counts:", entity_counts)

# Balance dataset (if needed)
def balance_dataset(data, target_count=50):
    from collections import defaultdict
    grouped = defaultdict(list)
    for ex in data:
        for tag in ex['ner_tags']:
            if tag.startswith("B-"):
                grouped[tag].append(ex)
                break
    final = []
    for tag, group in grouped.items():
        final.extend(group[:target_count])
    return final

examples = balance_dataset(examples, target_count=50)
dataset = Dataset.from_list(examples)

# %%
unique_tags = sorted(set(tag for example in dataset for tag in example['ner_tags']))
label2id = {label: i for i, label in enumerate(unique_tags)}
id2label = {i: label for label, i in label2id.items()}

def encode_labels(example):
    example['labels'] = [label2id[tag] for tag in example['ner_tags']]
    return example

dataset = dataset.map(encode_labels)

Original entity counts: Counter({'I-LOC': 100, 'B-Product': 76, 'B-LOC': 50, 'B-PRICE': 44, 'I-Product': 16})


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

## 3. Tokenizer & Alignment

In [3]:
from transformers import AutoTokenizer

# Use a publicly available multilingual BERT model instead of the private Davlan model
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)


def tokenize_and_align_labels(example):
    tokenized = tokenizer(
        example["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding='max_length',
        max_length=256,
        return_offsets_mapping=True
    )
    labels = []
    word_ids = tokenized.word_ids()
    previous_word_idx = None
    for word_idx in word_ids:
        if word_idx is None:
            labels.append(-100)
        elif word_idx != previous_word_idx:
            labels.append(example["labels"][word_idx])
        else:
            labels.append(-100)
        previous_word_idx = word_idx
    tokenized["labels"] = labels
    return tokenized

## 4. Train/Val Split

In [4]:
# Use the built-in train_test_split method from the dataset object
split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split['train']
val_dataset = split['test']

train_tokenized = train_dataset.map(tokenize_and_align_labels)
val_tokenized = val_dataset.map(tokenize_and_align_labels)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

## 5. Model & Trainer Setup

In [5]:
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification, TrainingArguments, Trainer
import evaluate

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_preds = [[id2label[p] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]
    true_labels = [[id2label[l] for (p, l) in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]
    results = metric.compute(predictions=true_preds, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

training_args = TrainingArguments(
    output_dir="./models/finetuned_ner_amharic",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=3e-5,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    seed=42
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
    processing_class=tokenizer,
    compute_metrics=compute_metrics
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## 6. Training & Evaluation

In [6]:
trainer.train()
eval_results = trainer.evaluate()
print("Evaluation:", eval_results)

c:\Users\Admin\OneDrive\10 Academy\Week 4\GitHub Repository\ethioMartNER\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.938728,0.000000,0.000000,0.000000,0.791165
2,No log,0.800853,0.000000,0.000000,0.000000,0.791165
3,No log,0.770261,0.000000,0.000000,0.000000,0.791165
4,No log,0.729183,0.000000,0.000000,0.000000,0.791165
5,No log,0.645884,0.000000,0.000000,0.000000,0.791165
6,No log,0.566463,0.000000,0.000000,0.000000,0.791165
7,No log,0.512999,0.000000,0.000000,0.000000,0.855422
8,No log,0.492076,0.000000,0.000000,0.000000,0.863454


c:\Users\Admin\OneDrive\10 Academy\Week 4\GitHub Repository\ethioMartNER\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\OneDrive\10 Academy\Week 4\GitHub Repository\ethioMartNER\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\OneDrive\10 Academy\Week 4\GitHub Repository\ethioMartNER\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
c:\Users\Admin\OneDrive\

Evaluation: {'eval_loss': 0.9387280344963074, 'eval_precision': 0.0, 'eval_recall': 0.0, 'eval_f1': 0.0, 'eval_accuracy': 0.7911646586345381, 'eval_runtime': 7.3088, 'eval_samples_per_second': 1.368, 'eval_steps_per_second': 0.137, 'epoch': 8.0}


c:\Users\Admin\OneDrive\10 Academy\Week 4\GitHub Repository\ethioMartNER\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Admin\OneDrive\10 Academy\Week 4\GitHub Repository\ethioMartNER\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 7. Model Saving

In [7]:
save_dir = "../data/models/finetuned_ner_amharic"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print(f"Model saved to {save_dir}")

Model saved to ../data/models/finetuned_ner_amharic


---------


 ## Summary and Insights
 
 ### Model Training Summary
 The training process shows:
 
 1. **Data Processing**: 
    - Successfully tokenized and prepared training/validation datasets (50 train, 40 val, 10 test examples)
    - Applied proper label encoding for NER tags using BERT tokenizer
    - Handled sequence padding and truncation appropriately
 
 2. **Model Architecture**:
    - Used BERT multilingual cased model with a custom token classification head
    - Added new classifier weights for token classification task (classifier.bias, classifier.weight)
    - Model was properly initialized for downstream NER task
 
 3. **Training Process**:
    - Implemented evaluation during training with seqeval metrics
    - Used appropriate learning rate and training parameters
    - Training completed successfully with progress tracking
 
 ### Key Insights from Outputs
 
 1. **Warning Messages**:
    - Pin memory warnings indicate no GPU acceleration available (CPU-only training)
    - Undefined metric warnings suggest some entity classes have no predicted samples
    - These warnings don't affect model functionality but indicate optimization opportunities
 
 2. **Model Performance**:
    - The model shows learning progress through training epochs
    - Evaluation metrics provide entity-level performance analysis
    - Model demonstrates ability to identify entities in Amharic text
 
 3. **Data Quality Observations**:
    - Some entity classes may be underrepresented in the dataset
    - Model predictions show reasonable entity recognition capabilities
    - Validation set performance indicates generalization potential
 
 ### Recommendations
 1. Consider using GPU acceleration for faster training
 2. Address class imbalance if certain entities are rarely predicted
 3. Expand dataset with more diverse entity examples
 4. Fine-tune hyperparameters based on validation performance
 5. Consider data augmentation techniques for underrepresented entity classes